# Cleaning Data Exercises

In this exercise, we'll be returning to the American Community Survey data we used previously to measuring racial income inequality in the United States. In today's exercise, we'll be using it to measure the returns to education and how those returns vary by race and gender.




## Gradescope Autograding

Please follow [all standard guidance](https://www.practicaldatascience.org/html/autograder_guidelines.html) for submitting this assignment to the Gradescope autograder, including storing your solutions in a dictionary called `results` and ensuring your notebook runs from the start to completion without any errors.

For this assignment, please name your file `exercise_cleaning.ipynb` before uploading.

You can check that you have answers for all questions in your `results` dictionary with this code:

```python
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}
```


### Submission Limits

Please remember that you are **only allowed three submissions to the autograder.** Your last submission (if you submit 3 or fewer times), or your third submission (if you submit more than 3 times) will determine your grade Submissions that error out will **not** count against this total.

## Exercises

### Exercise 1

For these cleaning exercises, we'll return to the ACS data we've used before one last time. We'll be working with `US_ACS_2017_10pct_sample.dta`. Import the data (please use url for the autograder).

In [21]:
import pandas as pd
import numpy as np

results = {}

acs = pd.read_stata(
    "https://github.com/nickeubank/MIDS_Data/raw/refs/heads/master/US_AmericanCommunitySurvey/US_ACS_2017_10pct_sample.dta"
)

pd.set_option("mode.copy_on_write", True)

### Exercise 2

For our exercises today, we'll focus on `age`, `sex`, `educ` (education), and `inctot` (total income). Subset your data to those variables, and quickly look at a sample of 10 rows.

In [22]:
acs = acs[["age", "sex", "educ", "inctot"]]
acs.sample(10)

,age,sex,educ,inctot
247761,72,female,1 year of college,41900
28892,65,female,2 years of college,38200
285598,17,female,grade 11,0
116494,55,male,5+ years of college,171000
135335,74,female,grade 12,2500
126573,34,male,grade 12,72000
198617,38,female,5+ years of college,398000
300054,51,female,4 years of college,114500
187858,21,male,2 years of college,12000
212277,43,male,4 years of college,105000


### Exercise 3

As before, all the values of `9999999` have the potential to cause us real problems, so replace all the values of `inctot` that are `9999999` with `np.nan`. 

In [23]:
acs.loc[acs["inctot"] == 9999999, "inctot"] = np.nan

### Exercise 4

Attempt to calculate the average age of people in our data. What do you get? Why are you getting that error?

You *should* get an error in trying to answer this question, but **PLEASE LEAVE THE CODE THAT GENERATES THIS ERROR COMMENTED OUT SO YOUR NOTEBOOK WILL RUN IN THE AUTOGRADER**. 

Then talk about the error in a markdown cell.

In [24]:
# acs["age"].mean()
# acs["age"].dtype

> I got this error when I attempted to calculate the average age of people in our data:   
> `TypeError: 'Categorical' with dtype category does not support reduction 'mean'`   
> It turns out that the "age" column has a categorical data type and that doesn't support the mean function. Thus, calling the mean function on the column throws the error.

### Exercise 5

We want to be able to calculate things using age, so we need it to be a numeric type. Check the current type of `age`, and look at all the values of `age` to figure out why it's categorical and not numeric. You should find two problematic categories. Store the values of these categories in `"ex5_age_young"` and `"ex5_age_old"` (once you find them, it should be clear which is which).

In [25]:
# print(acs_sub["age"].cat.categories)
print(acs.loc[~acs["age"].str.isnumeric(), "age"].unique())
results["ex5_age_young"] = ex5_age_young = "less than 1 year old"
results["ex5_age_old"] = ex5_age_old = "90 (90+ in 1980 and 1990)"
print(
    f"The two non-numeric values in the category are '{ex5_age_young}' and '{ex5_age_old}'."
)

['less than 1 year old', '90 (90+ in 1980 and 1990)']
Categories (97, object): ['less than 1 year old' < '1' < '2' < '3' ... '93' < '94' < '95' < '96']
The two non-numeric values in the category are 'less than 1 year old' and '90 (90+ in 1980 and 1990)'.


### Exercise 6

In order to convert `age` into a numeric variable, we need to replace those problematic entries with values that `pandas` can later convert into numbers. Pick appropriate substitutions for the existing values and replace the current values. 

**Hint 1:** Categorical variables act like strings, so you might want to use string methods! 

**Hint 2:** Remember that characters like parentheses, pluses, asterisks, etc. are special in Python strings, and you have to escape them if you want them to be interpreted literally!

**Hint 3:** Because the US Census has been conducted regularly for hundreds of years but exactly how the census has been conducted have occasionally changed, variables are sometimes coded in a way that might be interpreted in different ways for different census years. For example, hypothetically, one might write `90 (90+ in 1980 and 1990)` if the Censuses conducted in 1980 and 1990 used to top-code age at 90 (any values *over* 90 were just coded as 90), but more recent Censuses no longer top-coded age and recorded ages over 90 as the respondents actual age.

In [26]:
# Interestingly on the surface, calling .cat on a categorical column is like getting extra functions for the column, not really accessing the category table
acs["age"] = acs["age"].cat.rename_categories(
    {"less than 1 year old": "0", "90 (90+ in 1980 and 1990)": "90"}
)

### Exercise 7

Now convert age from a categorical to numeric. Calculate the average age amoung this group, and store it in `"ex7_avg_age"`.

In [27]:
acs["age"] = acs["age"].astype(int)
results["ex7_avg_age"] = ex7_avg_age = acs["age"].mean()
print(f"The average age is {ex7_avg_age:.1f} years.")

The average age is 41.3 years.


### Exercise 8

Let's now filter out anyone in our data whose age is less than 18. Note that before made `age` a numeric variable, we couldn't do this! Again, calculate the average age and this time store it in `"ex8_avg_age"`. 

Use this sample of people 18 and over for all subsequent exercises.

In [28]:
acs = acs.loc[acs["age"] >= 18]
results["ex8_avg_age"] = ex8_avg_age = acs["age"].mean()
print(f"The average age now is {ex8_avg_age:.1f} years.")

The average age now is 49.8 years.


### Exercise 9

Create an indicator variable for whether each person has *at least* a college Bachelor's degree called `college_degree`. Use this variable to calculate the number of people in the dataset with a college degree. You may assume that to get a college degree you need to complete at least 4 years of college. Save the result as `"ex9_num_college"`.

In [29]:
# acs["educ"].cat.categories
college_degree = acs["educ"].isin(["4 years of college", "5+ years of college"])
acs["college_degree"] = college_degree
results["ex9_num_college"] = ex9_num_college = len(acs.loc[acs["college_degree"]])
print(f"The number of people with a college degree are {ex9_num_college:,.0f}.")

The number of people with a college degree are 77,013.


### Exercise 10

Let's examine how the educational gender gap. Use `pd.crosstab` to create a cross-tabulation of `sex` and `college_degree`. `pd.crosstab` will give you the number of people who have each combination of `sex` and `college_degree` (so in this case, it will give us a 2x2 table with Male and Female as rows, and `college_degree` True and False as columns, or vice versa. 

In [30]:
acs_sex_degree = pd.crosstab(acs["sex"], acs["college_degree"])
acs_sex_degree

college_degree,False,True
sex,,
male,85821,36181
female,90200,40832


### Exercise 11

Counts are kind of hard to interpret. `pd.crosstab` can also normalize values to give percentages. Look at the `pd.crosstab` help file to figure out how to normalize the values in the table. Normalize them so that you get the share of men with and without college degree, and the share of women with and without college degrees.

Store the share (between 0 and 1) of men with college degrees in `"ex11_share_male_w_degrees"`, and the share of women with degrees in `"ex11_share_female_w_degrees"`.

In [31]:
acs_sex_degree = pd.crosstab(acs["sex"], acs["college_degree"], normalize="index")
results["ex11_share_male_w_degrees"] = ex11_share_male_w_degrees = acs_sex_degree.iloc[
    0, 1
]
results["ex11_share_female_w_degrees"] = ex11_share_female_w_degrees = (
    acs_sex_degree.iloc[1, 1]
)
print(f"The share of men with college degree is {ex11_share_male_w_degrees:.2%}.")
print(f"The share of women with college degree is {ex11_share_female_w_degrees:.2%}.")
acs_sex_degree

The share of men with college degree is 29.66%.
The share of women with college degree is 31.16%.


college_degree,False,True
sex,,
male,0.703439,0.296561
female,0.688381,0.311619


### Exercise 12

Now, let's recreate that table for people who are 40 and over and people under 40. Over time, what does this suggest about the absolute difference in the share of men and women earning college degrees? Has it gotten larger, stayed the same, or gotten smaller? Store your answer (either `"the absolute difference has increased"` or `"the absolute difference has decreased"`) in `"ex12_comparing"`.

In [37]:
acs_over_40 = acs.loc[acs["age"] >= 40]
acs_under_40 = acs.loc[acs["age"] < 40]
acs_cross_over_40 = pd.crosstab(
    acs_over_40["sex"], acs_over_40["college_degree"], normalize="columns"
)
acs_cross_under_40 = pd.crosstab(
    acs_under_40["sex"], acs_under_40["college_degree"], normalize="columns"
)
print(
    f"For people 40 years and above, the share of men with college degree is {acs_cross_over_40.iloc[0,1]:.2%} and the share of women is {acs_cross_over_40.iloc[1,1]:.2%}."
)
print(
    f"For people below 40 years, the share of men with college degree is {acs_cross_under_40.iloc[0,1]:.2%} and the share of women is {acs_cross_under_40.iloc[1,1]:.2%}."
)
results["ex12_comparing"] = ex12_comparing = "the absolute difference has increased"
print(f"Thus, {ex12_comparing} over time.")
print()
print("--------People 40 years and above------------")
print(acs_cross_over_40)
print()
print("--------People under 40 years------------")
print(acs_cross_under_40)

For people 40 years and above, the share of men with college degree is 48.56% and the share of women is 51.44%.
For people below 40 years, the share of men with college degree is 43.71% and the share of women is 56.29%.
Thus, the absolute difference has increased over time.

--------People 40 years and above------------
college_degree     False     True 
sex                               
male            0.465718  0.485587
female          0.534282  0.514413

--------People under 40 years------------
college_degree     False     True 
sex                               
male            0.530066  0.437058
female          0.469934  0.562942


### Exercise 13

In words, what is causing the change noted in Exercise 12 (i.e., looking at the tables above, tell me a story about Men and Women's College attainment).

In [49]:
counts_over_40 = acs_over_40["sex"].value_counts()
print("Gender counts 40 and over")
print(counts_over_40)
print()
print("Gender counts under 40")
counts_under_40 = acs_under_40["sex"].value_counts()
print(counts_under_40)
print()
counts_acs_cross_over_40 = pd.crosstab(
    acs_over_40["sex"], acs_over_40["college_degree"]
)
counts_acs_cross_under_40 = pd.crosstab(
    acs_under_40["sex"], acs_under_40["college_degree"]
)
print("Gender counts 40 and over attaining degree")
print(counts_acs_cross_over_40)
print()
print("Gender counts under 40 attaining degree")
counts_under_40 = acs_under_40["sex"].value_counts()
print(counts_acs_cross_under_40)

Gender counts 40 and over
sex
female    88853
male      79383
Name: count, dtype: int64

Gender counts under 40
sex
male      42619
female    42179
Name: count, dtype: int64

Gender counts 40 and over attaining degree
college_degree  False  True 
sex                         
male            54149  25234
female          62121  26732

Gender counts under 40 attaining degree
college_degree  False  True 
sex                         
male            31672  10947
female          28079  14100


The difference between the _number of surveyed males and females under 40 years_ has reduced in recent years, in favor of males, compared to the difference in those above 40 years. Thus, the recent increase in the number of females attaining degree is not caused by an increased number of surveyed females. 

From exercise 12, 49% of males above 40 years attained college degree and 44% of males under 40 years attained college degree. This is a drop in the relative number of males attaining college degree by about 5%. However, 51% of females above 40 years attained college degree and 56% of females under 40 years attained college degree. This is an increase in the relative number of females attaining college degree by about 5%. Thus, the recent gap increase in the _number of males and females attaining college degree_, in favor of females, is caused by _fewer males and more females_ attaining college degree.

In [33]:
assert set(results.keys()) == {
    "ex5_age_young",
    "ex5_age_old",
    "ex7_avg_age",
    "ex8_avg_age",
    "ex9_num_college",
    "ex11_share_male_w_degrees",
    "ex11_share_female_w_degrees",
    "ex12_comparing",
}

## Want More Practice?

Calculate the educational racial gap in the United States for White Americans, Black Americans, Hispanic Americans, and other groups. 

Note that to do these calculations, you'll have to deal with the fact that unlike most Americans, the American Census Bureau treats "Hispanic" not as a racial category, but a linguistic one. As a result, the racial category "White" in `race` actually includes most Hispanic Americans. For this analysis, we wish to work with the mutually exclusive categories of "White, non-Hispanic", "White, Hispanic", "Black (Hispanic or non-Hispanic)", and a category for everyone else. 